In [ ]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.8 MB/s eta 0:00:00


In [ ]:
#clean cuda
import torch
torch.cuda.empty_cache()

In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Input text
text = "Barack Obama was the president of the United States and lives in Washington."

# Process text
doc = nlp(text)

# Extract entities
entities = [(ent.text, ent.label_) for ent in doc.ents]
print(entities)

[('Barack Obama', 'PERSON'), ('the United States', 'GPE'), ('Washington', 'GPE')]


In [ ]:
import json
data_set = "/content/few-nerd.json"
entity_types_file = "/content/few-nerd_entity_types.json"
with open(data_set, "r") as file:
    data = json.load(file)
with open(entity_types_file, "r") as file:
    entity_types = json.load(file)

In [ ]:
def find_word_indices(words, entity):
    """
    Finds the word start and end indices of an entity in a given text.

    Parameters:
        text (str): The full input text.
        entity (str): The entity to find.

    Returns:
        tuple: (word_start_index, word_end_index) of the entity in the text, or (-1, -1) if not found.
    """
    entity_words = entity.split()

    # Loop through the words in the text to find the entity
    for i in range(len(words)):
        if words[i:i + len(entity_words)] == entity_words:
            word_start_index = i
            word_end_index = i + len(entity_words) - 1
            return word_start_index, word_end_index

    # Return (-1, -1) if the entity is not found
    return -1, -1

In [ ]:
import spacy
import json
from threading import Lock
from tqdm import tqdm
# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Lock for thread-safe file writing
file_lock = Lock()

def process_item(items):
    """
    Process a batch of items to perform NER using SpaCy and save results in a JSONL file.
    """
    inputs_text = []
    inputs_text_unified = []
    for item in items:
        input_text = item["tokenized_text"]
        inputs_text.append(input_text)
        input_text_unified = " ".join(input_text)
        inputs_text_unified.append(input_text_unified)

    # Perform NER with SpaCy
    answers = []
    for text in tqdm(inputs_text_unified):
        doc = nlp(text)
        ner_results = [{"entity": ent.text, "type": ent.label_} for ent in doc.ents]
        answers.append(ner_results)

    # Map SpaCy entity types to custom types
    entity_type_mapping = {
        "PERSON": "person",
        "WORK_OF_ART": "art",
        "PRODUCT": "product",
        "GPE": "location",
        "LOC": "location",
        "FAC": "building",
        "EVENT": "event",
        "ORG": "organization"
    }

    for i, answer in enumerate(answers):
        predictions = []
        if not answer:
            continue

        for entity in answer:
            # Map SpaCy entity type to custom type
            custom_type = entity_type_mapping.get(entity["type"], "other")

            # Find word indices
            start, end = find_word_indices(inputs_text[i], entity["entity"])
            predictions.append([start, end, custom_type])  # Append as [start, end, entity type]

        # Write results to the file with thread safety
        with file_lock:
            with open("results_spacy.jsonl", "a+") as file:
                json.dump({"item": items[i], "predictions": predictions}, file)
                file.write("\n")
process_item(data)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
 26%|██▌       | 9639/37648 [01:28<03:42, 125.99it/s]